In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import talib as ta

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
ohlcFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlcWithDelOI.csv"
yhfinanceFl = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\correctedEODDataYF.csv"

In [3]:
ohlc = pd.read_csv(ohlcFl)
ohlc["date"] = pd.to_datetime(ohlc.date).apply(lambda x: x.date())
yFin = pd.read_csv(yhfinanceFl)
yFin["Date"] = pd.to_datetime(yFin.Date, format="%Y%m%d").apply(lambda x: x.date())

In [4]:
ohlc.loc[ohlc.symbol == "NSE500", "date"].max()

datetime.date(2022, 4, 22)

In [5]:
yFin.rename(
    columns={
        "Date": "date",
        "Ticker": "symbol",
        "Close": "corrClose",
        "High": "corrHigh",
        "Low": "corrLow",
        "Open": "corrOpen",
        "Volume": "yfVolume",
    },
    inplace=True,
)

In [6]:
finalOHLC = pd.merge(
    left=yFin,
    right=ohlc,
    left_on=["symbol", "date"],
    right_on=["symbol", "date"],
    how="outer",
)

In [7]:
def getNifty500():

    nList = "nifty500"
    url = "https://archives.nseindia.com/content/indices/ind_" + nList + "list.csv"

    return pd.read_csv(url)


nifty500 = getNifty500()

In [8]:
tenYearAgo = datetime.date.today() - relativedelta(years=10)
finalOHLC10yr = finalOHLC.loc[finalOHLC.date >= tenYearAgo, :]

In [9]:
finalOHLC10yrNifty500 = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(nifty500.Symbol), :
].dropna(subset=["corrOpen", "corrClose", "corrHigh", "corrLow"])

In [10]:
finalOHLC10yrNifty500 = finalOHLC10yrNifty500[
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ]
]
finalOHLC10yrNifty500.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
    },
    inplace=True,
)
finalOHLC10yrNifty500.head()

,date,symbol,close,high,low,open,volume,delivery,OI,company,industry
839292,2012-04-23,3MINDIA,4059.649902,4269.950195,3955.000000,4130.000000,434.0,325.0,NaN,3M India Ltd.,Diversified
839293,2012-04-23,AARTIDRUGS,12.182687,12.677226,11.922123,12.230545,10310.0,7443.0,NaN,Aarti Drugs Ltd.,Healthcare
839294,2012-04-23,AARTIIND,14.151553,14.554114,14.048333,14.450894,33124.0,25224.0,NaN,Aarti Industries Ltd.,Chemicals
839295,2012-04-23,ABB,753.716309,772.466855,751.892677,771.438169,24241.0,9395.0,NaN,ABB India Ltd.,Capital Goods
839296,2012-04-23,ABBOTINDIA,1564.351807,1602.161788,1558.172216,1597.446967,3229.0,1615.0,NaN,Abbott India Ltd.,Healthcare


In [11]:
finalOHLC10yrNifty500.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [12]:
finalOHLC10yrIndex = finalOHLC10yr.loc[
    finalOHLC10yr.industry == "INDEX",
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrIndex.head()

,date,symbol,open,high,low,close,volume,delivery,OI,company,industry
1840345,2012-04-23,NSENIFTY,5277.40,5310.55,5187.15,5200.60,138042341.0,0.0,NaN,IND_NSENIFTY,INDEX
1840346,2012-04-24,NSENIFTY,5215.90,5232.35,5180.35,5222.65,155674654.0,0.0,24153050.0,IND_NSENIFTY,INDEX
1840347,2012-04-25,NSENIFTY,5222.20,5236.10,5160.65,5202.00,142910739.0,0.0,26554900.0,IND_NSENIFTY,INDEX
1840348,2012-04-26,NSENIFTY,5214.75,5215.60,5179.05,5189.00,185899437.0,0.0,26921600.0,IND_NSENIFTY,INDEX
1840349,2012-04-27,NSENIFTY,5189.00,5223.05,5154.30,5190.60,128759445.0,0.0,19245900.0,IND_NSENIFTY,INDEX


In [13]:
finalOHLC10yrIndex.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [14]:
finalOHLC10yrIndex.loc[finalOHLC10yrIndex.symbol == "NSE500", "date"].max()

datetime.date(2022, 4, 22)

In [15]:
othersImp = [
    "CRUDE_OIL",
    "GOLD",
    "HDFC_US_ADR",
    "ICICI_US_ADR",
    "INFOSYS_US_ADR",
    "USDINR",
    "DR_REDDY_US_ADR",
    "SILVER",
    "TATAMOTORS_US_ADR",
    "WIPRO_US_ADR",
]
finalOHLC10yrOther = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(othersImp),
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "yfVolume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrOther["company"] = "OTHERS"
finalOHLC10yrOther["industry"] = "OTHERS"
finalOHLC10yrOther.head()

,date,symbol,corrClose,corrHigh,corrLow,corrOpen,yfVolume,delivery,OI,company,industry
29782,2012-04-23,CRUDE_OIL,103.110001,103.900002,101.820000,103.820000,213202.0,NaN,NaN,OTHERS,OTHERS
29783,2012-04-23,GOLD,1631.900024,1642.400024,1625.099976,1642.400024,226.0,NaN,NaN,OTHERS,OTHERS
29784,2012-04-23,HDFC_US_ADR,15.555126,15.815619,15.424880,15.792361,1419400.0,NaN,NaN,OTHERS,OTHERS
29785,2012-04-23,ICICI_US_ADR,5.206793,5.213180,5.085444,5.117379,15341150.0,NaN,NaN,OTHERS,OTHERS
29786,2012-04-23,INFOSYS_US_ADR,4.157432,4.163805,4.047259,4.120100,22386400.0,NaN,NaN,OTHERS,OTHERS


In [16]:
finalOHLC10yrOther.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [17]:
finalOHLC10yrOther.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
        "yfVolume": "volume",
    },
    inplace=True,
)

In [18]:
corrDf = pd.DataFrame(columns=["stock", "testStock", "corr"])
for sym in finalOHLC10yrNifty500.symbol.unique():
    industry = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, "industry"
    ].values[0]
    stock = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, ["date", "symbol", "close"]
    ].sort_values(by=["date"])
    stock["ret"] = stock["close"].pct_change()
    stockNumOfDates = stock.date.shape[0]
    for testSym in finalOHLC10yrNifty500.loc[
        (
            (finalOHLC10yrNifty500.industry == industry)
            & (finalOHLC10yrNifty500.symbol != sym)
        ),
        :,
    ]["symbol"].unique():
        testStock = finalOHLC10yrNifty500.loc[
            finalOHLC10yrNifty500.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        noOfDates = testStock.shape[0]
        if stockNumOfDates == noOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    for testSym in finalOHLC10yrIndex.symbol.unique():
        testStock = finalOHLC10yrIndex.loc[
            finalOHLC10yrIndex.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
    for testSym in finalOHLC10yrOther.symbol.unique():
        testStock = finalOHLC10yrOther.loc[
            finalOHLC10yrOther.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()

            if (
                stock1["ret"].dropna().shape[0] > 3
                and testStock1["ret"].dropna().shape[0] > 3
            ):
                try:
                    corr, _ = pearsonr(
                        stock1["ret"].dropna().values, testStock1["ret"].dropna().values
                    )
                    corrDf = corrDf.append(
                        pd.DataFrame(
                            {"stock": [sym], "testStock": [testSym], "corr": [corr]}
                        )
                    )
                except:
                    continue

    # break

In [21]:
corrDf.sort_values(by=["corr"], ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\corr.csv",
    index=False,
)

In [18]:
allData = finalOHLC10yrOther.append(finalOHLC10yrIndex).append(finalOHLC10yrNifty500)
allData["date"] = allData.date.apply(lambda x: x.strftime("%Y%m%d"))
allData.fillna(0, inplace=True)
allData.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\nifty500OtherOHLC10yrs.csv",
    index=False,
)

In [19]:
allData.loc[allData.symbol == "TCS", "date"].max()

'20220422'

### Below code is for analysis stock by stock

In [71]:
symbol = "ABB"

In [72]:
import nsepy as npy
import datetime
from dateutil.relativedelta import relativedelta, TH
import re

start_date = datetime.date.today() - relativedelta(months=6)
end_date = datetime.date.today()

ohlc_data = npy.get_history(
    symbol=re.sub("&", "%26", symbol), start=start_date, end=end_date
).reset_index()

In [73]:
ohlc_data["vol_per_trade"] = ohlc_data["Volume"] / ohlc_data["Trades"]
ohlc_data.sort_values(by="Date", ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlc_data.csv",
    index=False,
)

In [74]:
ohlc_data.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,vol_per_trade
0,2021-10-25,ABB,EQ,1909.30,1920.00,1970.00,1830.00,1839.95,1850.20,1882.67,335087,6.308595e+13,27930,164241,0.4901,11.997386
1,2021-10-26,ABB,EQ,1850.20,1864.90,1868.85,1818.10,1847.00,1845.60,1845.76,206780,3.816672e+13,8473,161109,0.7791,24.404579
2,2021-10-27,ABB,EQ,1845.60,1920.05,1932.85,1866.05,1875.20,1879.70,1889.64,529236,1.000066e+14,45700,333461,0.6301,11.580656
3,2021-10-28,ABB,EQ,1879.70,1879.70,1999.00,1878.80,1990.00,1983.65,1971.58,1355503,2.672479e+14,58157,691843,0.5104,23.307650
4,2021-10-29,ABB,EQ,1983.65,1997.95,2153.75,1954.30,2095.20,2119.90,2045.66,960349,1.964547e+14,68610,345242,0.3595,13.997216


In [42]:
# symbol = symbol

In [43]:
mypath = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data"
holidays = pd.read_excel(
    mypath + os.path.sep + "NSEData" + os.path.sep + "nse_holidays.xlsx"
)
holidays.holidays = holidays.holidays.apply(lambda x: x.date())
# symbols = pd.read_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "FNO.csv")
today = datetime.datetime.today()
from_date = today - relativedelta(months=6)
dtList = list(pd.date_range(from_date, today, freq="B").to_pydatetime())
expriyDf = pd.DataFrame()
oiData = pd.DataFrame()
for dd in dtList:
    if holidays.holidays.isin([datetime.date(dd.year, dd.month, dd.day)]).sum() == 0:
        end_of_month = dd + relativedelta(day=31)
        expiry_date = end_of_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date.year, expiry_date.month, expiry_date.day
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date = expiry_date - relativedelta(day=1)
        end_of_next_month = dd + relativedelta(months=1, day=31)
        expiry_date_next_month = end_of_next_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_next_month.year,
                            expiry_date_next_month.month,
                            expiry_date_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_next_month = expiry_date_next_month - relativedelta(day=1)

        end_of_month_after_next_month = dd + relativedelta(months=2, day=31)
        expiry_date_month_after_next_month = (
            end_of_month_after_next_month + relativedelta(weekday=TH(-1))
        )
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_month_after_next_month.year,
                            expiry_date_month_after_next_month.month,
                            expiry_date_month_after_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_month_after_next_month = (
                    expiry_date_month_after_next_month - relativedelta(day=1)
                )
        if dd > expiry_date:
            expiry_date = expiry_date_next_month
            expiry_date_next_month = expiry_date_month_after_next_month
            end_of_month_after_next_month = dd + relativedelta(months=3, day=31)
            expiry_date_month_after_next_month = (
                end_of_month_after_next_month + relativedelta(weekday=TH(-1))
            )
            while True:
                if (
                    holidays.holidays.isin(
                        [
                            datetime.date(
                                expiry_date_month_after_next_month.year,
                                expiry_date_month_after_next_month.month,
                                expiry_date_month_after_next_month.day,
                            )
                        ]
                    ).sum()
                    == 0
                ):
                    break
                else:
                    expiry_date_month_after_next_month = (
                        expiry_date_month_after_next_month - relativedelta(day=1)
                    )

        expriyDf = expriyDf.append(
            pd.DataFrame(
                {
                    "date": [dd],
                    "current_exp_date": [expiry_date],
                    "next_month_exp_date": [expiry_date_next_month],
                    "month_after_next_month_exp_date": [
                        expiry_date_month_after_next_month
                    ],
                }
            )
        )
for _, row in expriyDf.iterrows():
    logging.info(
        "fetching features data for %s for date %s exp dates %s %s %s"
        % (
            symbol,
            datetime.datetime.strftime(row["date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["current_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(row["next_month_exp_date"].date(), "%Y-%m-%d"),
            datetime.datetime.strftime(
                row["month_after_next_month_exp_date"].date(), "%Y-%m-%d"
            ),
        )
    )
    current_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["current_exp_date"].date(),
    )
    next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["next_month_exp_date"].date(),
    )
    month_after_next_month_features = npy.get_history(
        symbol=symbol,
        start=row["date"].date(),
        end=row["date"].date(),
        futures=True,
        expiry_date=row["month_after_next_month_exp_date"].date(),
    )
    if (
        (len(current_month_features["Open Interest"].values) == 1)
        and (len(next_month_features["Open Interest"].values) > 0)
        and (len(month_after_next_month_features["Open Interest"].values) > 0)
    ):
        oiData = oiData.append(
            pd.DataFrame(
                {
                    "Symbol": [symbol],
                    "Date": [row["date"].date()],
                    "cummOI": [
                        current_month_features["Open Interest"].values[0]
                        + next_month_features["Open Interest"].values[0]
                        + month_after_next_month_features["Open Interest"].values[0]
                    ],
                }
            )
        )

In [44]:
ohlc_oi_date = pd.merge(
    left=oiData,
    right=ohlc_data,
    left_on=["Date", "Symbol"],
    right_on=["Date", "Symbol"],
    how="inner",
)

In [45]:
# ohlc_oi_date["vol_per_trade"] = ohlc_oi_date["Volume"] / ohlc_oi_date["Trades"]
ohlc_oi_date.sort_values(by="Date", ascending=False).to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlc_oi_date.csv",
    index=False,
)

In [46]:
ohlc_oi_date.set_index("Date", inplace=True)
ohlc_oi_date.sort_index(inplace=True)

In [47]:
ohlc_oi_date["5 Day avg Del Vol"] = ta.MA(
    ohlc_oi_date["Deliverable Volume"], 5, ta.MA_Type.SMA
).shift(1)

In [48]:
avgDelPct = ohlc_oi_date["%Deliverble"].mean()
avgVolPerTrade = ohlc_oi_date["vol_per_trade"].mean()

In [49]:
ohlc_oi_date["OI Chng"] = ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
ohlc_oi_date["% OI Change"] = (
    ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
) / ohlc_oi_date["cummOI"].shift(1)
ohlc_oi_date["% Price Change"] = (
    ohlc_oi_date["Close"] - ohlc_oi_date["Close"].shift(1)
) / ohlc_oi_date["Close"].shift(1)

In [50]:
def oiAnalysis(prcChng, oiChg):
    if prcChng > 0 and oiChg > 0:
        return "LongBuildUp"
    elif prcChng > 0 and oiChg < 0:
        return "ShortCovering"
    elif prcChng < 0 and oiChg > 0:
        return "ShortBuildUp"
    elif prcChng < 0 and oiChg < 0:
        return "LongUnWinding"


ohlc_oi_date["oiAnalysis"] = ohlc_oi_date.apply(
    lambda x: oiAnalysis(x["% Price Change"], x["% OI Change"]), axis=1
)

In [51]:
ohlc_oi_date.sort_index(ascending=False, inplace=True)

In [52]:
last5Days = 5
LBD_Days = sum(ohlc_oi_date.iloc[0:last5Days, :]["oiAnalysis"] == "LongBuildUp")
cls_higher_vwap = ohlc_oi_date.iloc[0, :]["Close"] > ohlc_oi_date.iloc[0, :]["VWAP"]
del_pct_higher_than_avg = ohlc_oi_date.iloc[0, :]["%Deliverble"] > avgDelPct
vol_per_trade_than_avg = ohlc_oi_date.iloc[0, :]["vol_per_trade"] > avgVolPerTrade

In [53]:
signal = "None"
if (
    del_pct_higher_than_avg
    and vol_per_trade_than_avg
    and cls_higher_vwap
    and LBD_Days > 1
    and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
):
    signal = "VERY_STRONG"
elif (
    del_pct_higher_than_avg
    and vol_per_trade_than_avg
    and cls_higher_vwap
    and LBD_Days == 1
    and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
):
    signal = "STRONG"
else:
    signal = "None"
print(signal)

None


### download FII data

In [ ]:
endDate = datetime.date.today()
startDate = endDate - relativedelta(months=6)
dtSrs = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%b%d%Y")
)
dtSrsLong = pd.Series(pd.date_range(startDate, endDate, freq="SM")).apply(
    lambda x: x.strftime("%B%d%Y")
)
dateDf = pd.DataFrame({"shrtDt": dtSrs, "longDt": dtSrsLong})
data = pd.DataFrame()

for _, dd in dateDf.iterrows():
    urlShrt = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["shrtDt"])
    )
    urlLng = (
        f"https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_%s.html"
        % (dd["longDt"])
    )
    print(urlShrt)
    if data.shape[0] == 0:
        try:
            data = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["shrtDt"]]

        except:
            data = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            data.columns = ["Sector", dd["longDt"]]

    else:
        try:
            temp = pd.read_html(urlShrt)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["shrtDt"]]
        except:
            temp = pd.read_html(urlLng)[0].iloc[3:, [1, 32]]
            temp.columns = ["Sector", dd["longDt"]]
        data = pd.merge(
            left=data, right=temp, left_on="Sector", right_on="Sector", how="outer"
        )

data.to_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "fii.csv", index=False)

### Find big move stocks


In [3]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import talib as ta

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
import nsepy as npy

logging.basicConfig(
    format="Date-Time : %(asctime)s : Line No. : %(lineno)d - %(message)s",
    level=logging.INFO,
)
monthsBack = 6


def oiAnalysis(prcChng, oiChg):
    if prcChng > 0 and oiChg > 0:
        return "LongBuildUp"
    elif prcChng > 0 and oiChg < 0:
        return "ShortCovering"
    elif prcChng < 0 and oiChg > 0:
        return "ShortBuildUp"
    elif prcChng < 0 and oiChg < 0:
        return "LongUnWinding"


analysis = pd.DataFrame(columns=["Symbol", "Signal"])
print("numpy version %s" % np.__version__)
start_date = datetime.date.today() - relativedelta(months=monthsBack)
end_date = datetime.date.today()

mypath = r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data"

holidays = pd.read_excel(
    mypath + os.path.sep + "NSEData" + os.path.sep + "nse_holidays.xlsx"
)
holidays.holidays = holidays.holidays.apply(lambda x: x.date())
symbols = pd.read_csv(mypath + os.path.sep + "NSEData" + os.path.sep + "FNO.csv")
today = end_date
from_date = start_date
dtList = list(pd.date_range(from_date, today, freq="B").to_pydatetime())
expriyDf = pd.DataFrame()

for dd in dtList:
    if holidays.holidays.isin([datetime.date(dd.year, dd.month, dd.day)]).sum() == 0:
        end_of_month = dd + relativedelta(day=31)
        expiry_date = end_of_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date.year, expiry_date.month, expiry_date.day
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date = expiry_date - relativedelta(day=1)
        end_of_next_month = dd + relativedelta(months=1, day=31)
        expiry_date_next_month = end_of_next_month + relativedelta(weekday=TH(-1))
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_next_month.year,
                            expiry_date_next_month.month,
                            expiry_date_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_next_month = expiry_date_next_month - relativedelta(day=1)

        end_of_month_after_next_month = dd + relativedelta(months=2, day=31)
        expiry_date_month_after_next_month = (
            end_of_month_after_next_month + relativedelta(weekday=TH(-1))
        )
        while True:
            if (
                holidays.holidays.isin(
                    [
                        datetime.date(
                            expiry_date_month_after_next_month.year,
                            expiry_date_month_after_next_month.month,
                            expiry_date_month_after_next_month.day,
                        )
                    ]
                ).sum()
                == 0
            ):
                break
            else:
                expiry_date_month_after_next_month = (
                    expiry_date_month_after_next_month - relativedelta(day=1)
                )
        if dd > expiry_date:
            expiry_date = expiry_date_next_month
            expiry_date_next_month = expiry_date_month_after_next_month
            end_of_month_after_next_month = dd + relativedelta(months=3, day=31)
            expiry_date_month_after_next_month = (
                end_of_month_after_next_month + relativedelta(weekday=TH(-1))
            )
            while True:
                if (
                    holidays.holidays.isin(
                        [
                            datetime.date(
                                expiry_date_month_after_next_month.year,
                                expiry_date_month_after_next_month.month,
                                expiry_date_month_after_next_month.day,
                            )
                        ]
                    ).sum()
                    == 0
                ):
                    break
                else:
                    expiry_date_month_after_next_month = (
                        expiry_date_month_after_next_month - relativedelta(day=1)
                    )

        expriyDf = expriyDf.append(
            pd.DataFrame(
                {
                    "date": [dd],
                    "current_exp_date": [expiry_date],
                    "next_month_exp_date": [expiry_date_next_month],
                    "month_after_next_month_exp_date": [
                        expiry_date_month_after_next_month
                    ],
                }
            )
        )

numpy version 1.20.1


In [4]:
allOI_data = pd.DataFrame()
for symbol in pd.read_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\WatchList\fno_list.tls",
    header=None,
).values:
    # if symbol[0].strip()=='TCS' :
    try:
        try:
            ohlc_oi_date_existing = pd.read_csv(
                r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\OHLC_VWAP_OI\ohlc_oi_date_"
                + symbol[0]
                + ".csv"
            )
            ohlc_oi_date_existing.Date = ohlc_oi_date_existing.Date.apply(
                lambda x: datetime.datetime.strptime(x, "%Y-%m-%d")
            )
            ohlc_oi_date_existing.set_index("Date", inplace=True)
            existing_end_date = ohlc_oi_date_existing.index.date.max() + relativedelta(
                days=1
            )
        except:
            ohlc_oi_date_existing = pd.DataFrame()
            existing_end_date = start_date
        # existing_end_date = datetime.date(2022,3,30)
        if existing_end_date <= end_date:
            ohlc_data = npy.get_history(
                symbol=re.sub("&", "%26", symbol[0]),
                start=existing_end_date,
                end=end_date,
            ).reset_index()
        else:
            ohlc_data = pd.DataFrame()
        if ohlc_data.shape[0] >= 1:

            ohlc_data["vol_per_trade"] = ohlc_data["Volume"] / ohlc_data["Trades"]
            oiData = pd.DataFrame()
            expriyDf = expriyDf.loc[
                expriyDf.date.apply(lambda x: x.date()) >= existing_end_date, :
            ]

            for _, row in expriyDf.iterrows():
                logging.info(
                    "run for %s  date %s exp date %s"
                    % (
                        symbol[0],
                        datetime.date.strftime(row["date"].date(), "%Y-%m-%d"),
                        datetime.date.strftime(
                            row["current_exp_date"].date(), "%Y-%m-%d"
                        ),
                    )
                )
                current_month_features = npy.get_history(
                    symbol=re.sub("&", "%26", symbol[0]),
                    start=row["date"].date(),
                    end=row["date"].date(),
                    futures=True,
                    expiry_date=row["current_exp_date"].date(),
                )
                next_month_features = npy.get_history(
                    symbol=re.sub("&", "%26", symbol[0]),
                    start=row["date"].date(),
                    end=row["date"].date(),
                    futures=True,
                    expiry_date=row["next_month_exp_date"].date(),
                )
                month_after_next_month_features = npy.get_history(
                    symbol=re.sub("&", "%26", symbol[0]),
                    start=row["date"].date(),
                    end=row["date"].date(),
                    futures=True,
                    expiry_date=row["month_after_next_month_exp_date"].date(),
                )
                # print(current_month_features.shape)
                if (
                    (len(current_month_features["Open Interest"].values) == 1)
                    and (len(next_month_features["Open Interest"].values) > 0)
                    and (
                        len(month_after_next_month_features["Open Interest"].values) > 0
                    )
                ):
                    oiData = oiData.append(
                        pd.DataFrame(
                            {
                                "Symbol": [symbol[0]],
                                "Date": [row["date"].date()],
                                "cummOI": [
                                    current_month_features["Open Interest"].values[0]
                                    + next_month_features["Open Interest"].values[0]
                                    + month_after_next_month_features[
                                        "Open Interest"
                                    ].values[0]
                                ],
                            }
                        )
                    )
            if oiData.shape[0] >= 1:
                ohlc_oi_date = pd.merge(
                    left=oiData,
                    right=ohlc_data,
                    left_on=["Date", "Symbol"],
                    right_on=["Date", "Symbol"],
                    how="inner",
                )
            if ohlc_oi_date_existing.shape[0] > 0:
                ohlc_oi_date_existing = (
                    ohlc_oi_date_existing.loc[
                        (
                            (ohlc_oi_date_existing.index.date < existing_end_date)
                            & (ohlc_oi_date_existing.index.date >= start_date)
                        ),
                        :,
                    ]
                    .reset_index()
                    .loc[:, list(ohlc_oi_date.columns)]
                )

                ohlc_oi_date = ohlc_oi_date.append(ohlc_oi_date_existing)
            ohlc_oi_date.set_index("Date", inplace=True)
            ohlc_oi_date.sort_index(inplace=True)

            ohlc_oi_date["5 Day avg Close Price"] = ta.EMA(
                ohlc_oi_date["Close"], 5
            ).shift(1)
            ohlc_oi_date["13 Day avg Close"] = ta.EMA(ohlc_oi_date["Close"], 13).shift(
                1
            )
            ohlc_oi_date["21 Day avg Close"] = ta.EMA(ohlc_oi_date["Close"], 21).shift(
                1
            )

            ohlc_oi_date["5 Day avg Del Vol"] = ta.MA(
                ohlc_oi_date["Deliverable Volume"], 5, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["13 Day avg Del Vol"] = ta.MA(
                ohlc_oi_date["Deliverable Volume"], 13, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["21 Day avg Del Vol"] = ta.MA(
                ohlc_oi_date["Deliverable Volume"], 21, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["5 Day avg vol_per_trade"] = ta.MA(
                ohlc_oi_date["vol_per_trade"], 5, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["13 Day avg vol_per_trade"] = ta.MA(
                ohlc_oi_date["vol_per_trade"], 13, ta.MA_Type.SMA
            ).shift(1)
            ohlc_oi_date["21 Day avg vol_per_trade"] = ta.MA(
                ohlc_oi_date["vol_per_trade"], 21, ta.MA_Type.SMA
            ).shift(1)

            ohlc_oi_date["isHAMMER"] = (
                ta.CDLHAMMER(
                    ohlc_oi_date.Open,
                    ohlc_oi_date.High,
                    ohlc_oi_date.Low,
                    ohlc_oi_date.Close,
                )
                == 100
            )
            ohlc_oi_date["isHARAMI"] = (
                ta.CDLHARAMI(
                    ohlc_oi_date.Open,
                    ohlc_oi_date.High,
                    ohlc_oi_date.Low,
                    ohlc_oi_date.Close,
                )
                == 100
            )

            avgDelPct = ohlc_oi_date["%Deliverble"].mean()
            avgVolPerTrade = ohlc_oi_date["vol_per_trade"].mean()
            ohlc_oi_date["OI Chng"] = ohlc_oi_date["cummOI"] - ohlc_oi_date[
                "cummOI"
            ].shift(1)
            ohlc_oi_date["% OI Change"] = (
                ohlc_oi_date["cummOI"] - ohlc_oi_date["cummOI"].shift(1)
            ) / ohlc_oi_date["cummOI"].shift(1)
            ohlc_oi_date["% Price Change"] = (
                ohlc_oi_date["Close"] - ohlc_oi_date["Close"].shift(1)
            ) / ohlc_oi_date["Close"].shift(1)
            ohlc_oi_date["oiAnalysis"] = ohlc_oi_date.apply(
                lambda x: oiAnalysis(x["% Price Change"], x["% OI Change"]), axis=1
            )
            ohlc_oi_date.sort_index(ascending=False, inplace=True)
            last5Days = 5
            LBD_Days = sum(
                ohlc_oi_date.iloc[0:last5Days, :]["oiAnalysis"] == "LongBuildUp"
            )
            cls_higher_vwap = (
                ohlc_oi_date.iloc[0, :]["Close"] > ohlc_oi_date.iloc[0, :]["VWAP"]
            )
            del_pct_higher_than_avg = ohlc_oi_date.iloc[0, :]["%Deliverble"] > avgDelPct
            vol_per_trade_than_avg = (
                ohlc_oi_date.iloc[0, :]["vol_per_trade"] > avgVolPerTrade
            )
            signal = "None"
            if (
                del_pct_higher_than_avg
                and vol_per_trade_than_avg
                and cls_higher_vwap
                and LBD_Days > 1
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "VERY_STRONG"
            elif (
                del_pct_higher_than_avg
                and vol_per_trade_than_avg
                and cls_higher_vwap
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "STRONG"
            elif (
                del_pct_higher_than_avg
                and vol_per_trade_than_avg
                and LBD_Days > 1
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "STRONG_NO_VWAP"
            elif (
                cls_higher_vwap
                and LBD_Days > 1
                and ohlc_oi_date.iloc[0, :]["oiAnalysis"] == "LongBuildUp"
            ):
                signal = "STRONG_NO_DEL"
            else:
                signal = ohlc_oi_date.iloc[0, :]["oiAnalysis"]
            analysis = analysis.append(
                pd.DataFrame({"Symbol": [symbol[0]], "Signal": [signal]})
            )
            ohlc_oi_date = ohlc_oi_date.sort_index(ascending=False).reset_index()
            ohlc_oi_date["Date"] = ohlc_oi_date["Date"].apply(
                lambda x: datetime.datetime.strftime(x, "%Y-%m-%d")
            )
            ohlc_oi_date = ohlc_oi_date.set_index("Date")
            ohlc_oi_date.to_csv(
                r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\OHLC_VWAP_OI\ohlc_oi_date_"
                + symbol[0]
                + ".csv"
            )

            allOI_data = allOI_data.append(ohlc_oi_date)
            analysis.to_csv(
                r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\bigMoveAnalysis.csv",
                index=False,
            )
            logging.info("Symbol %s analysis %s" % (symbol[0], signal))
        elif ohlc_oi_date_existing.shape[0] > 0:
            allOI_data = allOI_data.append(ohlc_oi_date_existing)
    except:
        logging.error("error ")
allOI_data.to_csv(
    r"C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\allOI_data_vwap.csv"
)

Date-Time : 2022-05-07 09:40:35,800 : Line No. : 157 - NumExpr defaulting to 8 threads.
Date-Time : 2022-05-07 09:40:37,973 : Line No. : 42 - run for APLLTD  date 2022-04-29 exp date 2022-05-26
Date-Time : 2022-05-07 09:40:38,792 : Line No. : 42 - run for APLLTD  date 2022-05-02 exp date 2022-05-26
Date-Time : 2022-05-07 09:40:39,214 : Line No. : 42 - run for APLLTD  date 2022-05-04 exp date 2022-05-26
Date-Time : 2022-05-07 09:40:39,412 : Line No. : 42 - run for APLLTD  date 2022-05-05 exp date 2022-05-26
Date-Time : 2022-05-07 09:40:39,643 : Line No. : 42 - run for APLLTD  date 2022-05-06 exp date 2022-05-26
Date-Time : 2022-05-07 09:40:39,881 : Line No. : 249 - error 
Date-Time : 2022-05-07 09:41:12,571 : Line No. : 42 - run for PFIZER  date 2022-04-29 exp date 2022-05-26
Date-Time : 2022-05-07 09:41:12,743 : Line No. : 42 - run for PFIZER  date 2022-05-02 exp date 2022-05-26
Date-Time : 2022-05-07 09:41:12,943 : Line No. : 42 - run for PFIZER  date 2022-05-04 exp date 2022-05-26
Da

In [14]:
logging.info(
    "run for %s  date %s exp date %s"
    % (
        symbols[0],
        datetime.date.strftime(row["date"].date(), "%Y-%m-%d"),
        datetime.date.strftime(row["current_exp_date"].date(), "%Y-%m-%d"),
    )
)


KeyError: 0

In [20]:
import pandas as pd
data = pd.read_csv(r'C:\Users\ksdee\Documents\PersonalFinance\Trading\Trading_Data\NSEData\ohlcWithDelOI.csv')
data.columns

Index(['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'delivery',
       'OI', 'company', 'Index'],
      dtype='object')

In [2]:
data.date.min(),data.date.max()

('2012-01-02', '2022-05-02 00:00:00')

In [3]:
data = data.dropna(subset=['OI']).sort_values(by=['date'],ascending=False)
data.loc[data.delivery>0,:].date.unique()

array(['2022-04-29', '2022-04-28', '2022-04-27', ..., '2012-01-09',
       '2012-01-05', '2012-01-03'], dtype=object)

In [10]:
data.loc[data.OI>0,:].date.min(),data.loc[data.symbol=='NSEMIDCAP150',:].date.max()

('2019-01-14', '2022-05-02 00:00:00')

In [21]:
import datetime
from dateutil.relativedelta import relativedelta, TH
data['date'] = data['date'].apply(lambda x : datetime.datetime.date(datetime.datetime.strptime(str(x)[0:10],'%Y-%m-%d')))
today=datetime.datetime.date(datetime.datetime.today())
twoYrBack = today - relativedelta(years=2)


array(['AARTIIND', 'ABB', 'ABBOTINDIA', 'ACC', 'ADANIENT', 'AMARAJABAT',
       'AMBUJACEM', 'APLLTD', 'APOLLOHOSP', 'APOLLOTYRE', 'ASHOKLEY',
       'ASIANPAINT', 'ASTRAL', 'ATUL', 'AUROPHARMA', 'AXISBANK',
       'BAJAJFINSV', 'BAJFINANCE', 'BALKRISIND', 'BALRAMCHIN',
       'BANKBARODA', 'BATAINDIA', 'BEL', 'BERGEPAINT', 'BHARATFORG',
       'BHARTIARTL', 'BHEL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA',
       'ZYDUSLIFE', 'CANBK', 'CANFINHOME', 'CENTURYTEX', 'CESC',
       'CHAMBLFERT', 'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL', 'CONCOR',
       'COROMANDEL', 'CUB', 'CUMMINSIND', 'DABUR', 'DEEPAKNTR',
       'DELTACORP', 'DIVISLAB', 'DLF', 'DRREDDY', 'EICHERMOT', 'ESCORTS',
       'EXIDEIND', 'FEDERALBNK', 'FSL', 'GAIL', 'GLENMARK', 'GNFC',
       'GODREJCP', 'GODREJPROP', 'GRANULES', 'GRASIM', 'GSPL', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HEROMOTOCO', 'HINDALCO',
       'HINDCOPPER', 'HINDPETRO', 'HINDUNILVR', 'HONAUT', 'ICICIBANK',
       'IDEA', 'IDFC', 'IGL', 'IN

In [22]:
data.loc[((data.OI>0)&(data.date>=twoYrBack)&(data.date<=today)) ,'symbol'].unique().shape

(200,)

In [15]:
today,twoYrBack

(datetime.date(2022, 5, 3), datetime.date(2020, 5, 3))

In [16]:
data.columns

Index(['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'delivery',
       'OI', 'company', 'Index'],
      dtype='object')